In [328]:
import pandas as pd
import numpy as np 
from string import ascii_letters
import time 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [329]:
header_text = "age,albumin_apache,apache_2_diagnosis,bmi,elective_surgery,encounter_id,ethnicity,gender,height,hospital_admit_source,hospital_death,hospital_id,icu_admit_source,icu_id,icu_stay_type,icu_type,patient_id,pre_icu_los_days,readmission_status,weight"
header = header_text.split(',')

In [330]:
results_path="./Results/model_HIVAE_inputDropout_ICU_Missing20_1_z2_y5_s10_batch100/model_HIVAE_inputDropout_ICU_Missing20_1_z2_y5_s10_batch100_data_reconstruction.csv"
input_path="./ICU/test_data_preprocessed_cleaned.csv"
train_path="./ICU/training_data_preprocessed_cleaned.csv"


In [331]:
reconstructed = pd.read_csv(results_path, names=header)
input_data = pd.read_csv(input_path, names=header)
train_data = pd.read_csv(train_path, names=header)

In [333]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)
len(reconstructed.columns)
len(input_data.columns)
reconstructed.describe()



,age,albumin_apache,apache_2_diagnosis,bmi,elective_surgery,encounter_id,ethnicity,gender,height,hospital_admit_source,hospital_death,hospital_id,icu_admit_source,icu_id,icu_stay_type,icu_type,patient_id,pre_icu_los_days,readmission_status,weight
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000
mean,62.500067,2.784062,23.463000,29.352199,0.179400,66156.13560,3.004400,1.530400,169.417787,5.142600,1.0,106.682000,1.684600,511.723800,0.119000,4.283200,66519.541200,0.856874,0.0,84.178196
std,16.226405,0.449915,12.822976,8.212316,0.383725,37483.61159,0.962999,0.499125,10.803868,2.855223,0.0,62.510239,0.937496,229.047648,0.469556,1.808328,37718.031697,2.500555,0.0,24.614695
min,16.000000,1.200000,1.000000,14.844926,0.000000,20.00000,1.000000,1.000000,137.200000,1.000000,1.0,2.000000,1.000000,82.000000,0.000000,0.000000,14.000000,-13.775000,0.0,38.600000
25%,53.000000,2.682000,12.000000,23.813362,0.000000,33574.75000,3.000000,1.000000,162.500000,4.000000,1.0,51.000000,1.000000,374.000000,0.000000,4.000000,33743.750000,0.035417,0.0,67.800000
50%,64.000000,2.709000,21.000000,27.891588,0.000000,66543.50000,3.000000,2.000000,170.000000,4.000000,1.0,112.000000,1.000000,504.000000,0.000000,5.000000,67128.500000,0.143056,0.0,80.000000
75%,75.000000,2.774000,37.000000,33.178803,0.000000,98807.25000,3.000000,2.000000,177.800000,5.000000,1.0,161.000000,2.000000,683.000000,0.000000,5.000000,99322.000000,0.416146,0.0,97.200000
max,89.000000,4.600000,44.000000,67.814990,1.000000,131048.00000,6.000000,2.000000,195.590000,15.000000,1.0,204.000000,5.000000,927.000000,2.000000,7.000000,131001.000000,48.838194,0.0,186.000000


In [334]:
input_data.describe()

,age,albumin_apache,apache_2_diagnosis,bmi,elective_surgery,encounter_id,ethnicity,gender,height,hospital_admit_source,hospital_death,hospital_id,icu_admit_source,icu_id,icu_stay_type,icu_type,patient_id,pre_icu_los_days,readmission_status,weight
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.00000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000
mean,59.856800,1.181680,183.240000,28.267992,0.179400,66156.13560,2.960600,1.52880,166.985928,4.209000,0.0,106.682000,1.683600,511.723800,1.119000,5.283200,66519.541200,0.856874,0.0,81.854812
std,20.982887,1.490396,89.465126,9.961649,0.383725,37483.61159,1.028231,0.50082,22.895383,3.626231,0.0,62.510239,0.938758,229.047648,0.469556,1.808328,37718.031697,2.500555,0.0,28.485507
min,0.000000,0.000000,0.000000,0.000000,0.000000,20.00000,0.000000,0.00000,0.000000,0.000000,0.0,2.000000,0.000000,82.000000,1.000000,1.000000,14.000000,-13.775000,0.0,0.000000
25%,51.000000,0.000000,113.000000,23.079175,0.000000,33574.75000,3.000000,1.00000,160.020000,1.000000,0.0,51.000000,1.000000,374.000000,1.000000,5.000000,33743.750000,0.035417,0.0,65.800000
50%,64.000000,0.000000,122.000000,27.491831,0.000000,66543.50000,3.000000,2.00000,170.000000,4.000000,0.0,112.000000,1.000000,504.000000,1.000000,6.000000,67128.500000,0.143056,0.0,79.800000
75%,75.000000,2.700000,301.000000,33.161683,0.000000,98807.25000,3.000000,2.00000,177.800000,5.000000,0.0,161.000000,2.000000,683.000000,1.000000,6.000000,99322.000000,0.416146,0.0,97.200000
max,89.000000,4.600000,308.000000,67.814990,1.000000,131048.00000,6.000000,2.00000,195.590000,15.000000,0.0,204.000000,5.000000,927.000000,3.000000,8.000000,131001.000000,48.838194,0.0,186.000000


In [335]:
reconstructed['hospital_death'].sum()

5000.0

In [336]:
targets = pd.read_csv('./ICU/test_target.csv')
targets['hospital_death'].sum()

425

In [337]:
y_pred = reconstructed['hospital_death']
y_true = targets['hospital_death']
print('Accuracy:', accuracy_score(y_true, y_pred))
roc_auc_score(y_true, y_pred)

Accuracy: 0.085


0.5

In [143]:
true_data['hospital_death'].sum()


,age,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bmi,bun_apache,creatinine_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,elective_surgery,encounter_id,ethnicity,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,gender,glucose_apache,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,heart_rate_apache,height,hematocrit_apache,hospital_admit_source,hospital_death,hospital_id,icu_admit_source,icu_id,icu_stay_type,icu_type,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,patient_id,ph_apache,pre_icu_los_days,readmission_status,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,weight
count,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.00000,5000.000000,5000.0000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,58.75120,1.034308,23.193800,40.260600,0.223800,0.03160,0.456918,28.130864,21.156400,1.239414,24.245200,14.315800,88.645200,49.488200,87.746000,48.857600,103.857800,70.353000,34.72200,18.613000,102.924133,63.243200,101.218933,62.133800,28.262600,12.196000,98.880000,89.829600,46.006600,28.073000,147.255600,95.777400,145.370000,94.487580,35.138410,34.111681,0.201000,8189.786200,2.918600,0.152921,4.432800,6.338600,0.013600,4.973800,1.543200,142.691840,15.344000,12.603000,71.84020,58.711400,68.192400,55.571600,89.970200,81.257600,99.462400,166.273210,26.350940,3.533000,0.0,107.89240,1.702800,1020.8630,0.010000,4.34680,0.177000,85.385732,10.708620,10.708620,33.797660,65783.633200,1.869278,0.873511,0.0,23.539360,114.833600,33.150708,1178.946087,0.354000,9.743754,81.567104
std,20.16682,1.424220,12.939572,26.950472,0.416831,0.17495,1.548905,9.320732,21.694904,1.589729,38.510166,22.822082,21.438443,13.702825,23.455326,14.711091,22.891168,17.905298,59.52283,29.852113,21.870949,15.249425,25.321195,17.192446,10.665495,5.159468,7.026655,11.784886,71.719702,44.605022,27.431529,21.558497,31.759138,23.897038,8.828311,8.582126,0.400788,4734.230845,0.999086,0.293734,1.079313,1.540527,0.115835,1.635986,0.498581,99.346614,29.745793,24.596796,24.40252,20.537333,29.256811,24.337998,27.587713,25.447026,32.117585,25.353184,14.444529,2.856386,0.0,65.03869,0.939068,50.1172,0.132301,1.81295,0.381707,42.308818,19.256762,19.256762,72.362465,38303.818254,3.200863,2.464539,0.0,15.149986,51.762108,10.466330,1478.990758,0.478257,8.167493,27.214119
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [98]:
wine_reconstruction = "/Users/viljamiraiski/Skolan/MAISTERI/SGPM/Project/HI-VAE/Results/model_HIVAE_inputDropout_Wine_Missing20_1_z2_y5_s10_batch100/model_HIVAE_inputDropout_Wine_Missing20_1_z2_y5_s10_batch100_data_reconstruction.csv"
wine_true = "/Users/viljamiraiski/Skolan/MAISTERI/SGPM/Project/HI-VAE/Results/model_HIVAE_inputDropout_Wine_Missing20_1_z2_y5_s10_batch100/model_HIVAE_inputDropout_Wine_Missing20_1_z2_y5_s10_batch100_data_true.csv"
wine_data_path = "./Wine/data.csv"

In [99]:
wine_r = pd.read_csv(wine_reconstruction)
wine_t = pd.read_csv(wine_true)
wine_data = pd.read_csv(wine_data_path)

In [93]:
wine_r.describe()

,7.314,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6.0,1.0
count,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000
mean,7.183180,0.334550,0.316580,5.100962,0.055648,28.892397,112.025870,0.994717,3.218581,0.529563,10.461928,5.720443,0.751847
std,1.202283,0.154594,0.135522,4.552715,0.031486,16.882208,55.664648,0.002847,0.148926,0.138826,1.102098,0.832401,0.431974
min,3.800000,0.080000,0.001000,0.600000,0.009000,1.000000,6.000000,0.987130,2.740000,0.220000,8.000000,3.000000,0.000000
25%,6.500000,0.240000,0.260000,1.800000,0.039000,15.000000,77.000000,0.992400,3.120000,0.440000,9.538500,5.000000,1.000000
50%,6.986000,0.280000,0.312000,2.500000,0.048000,26.992500,113.463000,0.995000,3.200000,0.506000,10.317000,6.000000,1.000000
75%,7.500000,0.390000,0.380000,8.100000,0.066000,38.000000,151.169750,0.997000,3.310000,0.590000,11.100000,6.000000,1.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.039000,4.010000,2.000000,14.900000,9.000000,1.000000


In [101]:
wine_r.head()

,7.314,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6.0,1.0
0,6.300,0.249,0.304,1.715,0.049,26.012,112.216,0.9940,3.30,0.49,10.862,6.0,1.0
1,8.100,0.327,0.400,2.452,0.050,30.000,97.000,0.9950,3.26,0.44,10.100,5.0,1.0
2,7.200,0.230,0.320,8.500,0.058,47.000,186.000,0.9956,3.19,0.40,9.900,6.0,1.0
3,6.762,0.266,0.320,8.500,0.058,47.000,186.000,0.9960,3.19,0.40,9.900,6.0,1.0
4,8.100,0.280,0.400,2.285,0.050,30.000,97.000,0.9951,3.26,0.44,10.100,6.0,1.0


In [100]:
wine_data.head()

,7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6,2
0,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,2
1,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,2
2,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,2
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,2
4,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,2


In [210]:
len(pd.Series([0,1,1,None]).unique())

3

In [247]:
a = pd.Series([58]*4601, dtype='int')
b = pd.Series(np.arange(4601))+1
df  = pd.DataFrame(a,b)
df.to_csv('Missingxx_y.csv')

In [282]:
datas = pd.read_csv('./spam/data.1.csv', names=np.arange(58))

In [286]:
datas[57]=0

In [287]:
datas

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.64,0.00,0.00,0.00,0.32,0.00,1.29,1.93,0.00,0.96,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,0
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0.00,1.59,0.0,0.43,0.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,0
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0.00,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0.0,1.16,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.12,0.00,0.06,0.06,0.0,0.0,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,0
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,0
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,0.00,1.88,0.00,0.00,0.00,0.00,0.00,0.00,0.62,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.31,0.31,0.31,0.0,0.0,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.00,0.00,2.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,2.00,0.0,0.0,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.80,0.30,0.00,0.00,0.00,0.00,0.90,1.50,0.00,0.30,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,1.20,0.0,0.0,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.00,0.00,0.00,1.93,0.00,0.32,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.32,0.00,0.32,0.0,0.0,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [290]:
datas.to_csv('data_test.csv', index=False)